# Implementing Univariate Recurrent Neural Networks (RNN)

### 1. Introduction and data gathering

- The aim is to implement a model to predict stock prices (rather returns) because prices are closer to be interpreted as sequences than returns. This makes them potentially more predictive using RNNsn than returns.

In [12]:
import numpy as np
import pandas as pd
import yfinance as yf  
from sklearn.preprocessing import MinMaxScaler

- Gathering data and defining outputs:

In [25]:
df = yf.download("AAPL", start="2010-01-01", end="2022-04-11")
del df["Volume"]
df["Output"] = df.Close.shift(-1)
df.reset_index(inplace=True)

[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['AAPL']: JSONDecodeError('Expecting value: line 1 column 1 (char 0)')


In [22]:
print("Length of data frame:", len(df))
df.head()

Length of data frame: 0


,Date,Open,High,Low,Close,Adj Close,Output
